Note: for using the widgets inside jupyterlab, you need an appropriate environment with nodejs and the widegetextensions install. 
In order to develop the notebook, the following environment was used:

- conda create -n secanalyzing python==3.10
- conda activate secanalyzing
- pip install jupyterlab
- jupyter labextension install @jupyter-widgets/jupyterlab-manager
- pip install secfsdstools
- pip install ipywidgets

checkout: https://gist.github.com/mwouts/a2de16feb90d33fd89334fb09f62742f
https://www.linkedin.com/pulse/interactive-dataframes-jupyter-google-colab-vs-code-pycharm-wouts/?trk=articles_directory
https://pbpython.com/dataframe-gui-overview.html

In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
from secfsdstools.e_read.searching import IndexSearch
from secfsdstools.e_read.companyreading import CompanyReader
from secfsdstools.e_read.reportreading import ReportReader
from secfsdstools.e_read.zipreportreading import ZipReportReader
from secfsdstools.d_index.indexing import ReportZipIndexer


# initialize the search class
search = IndexSearch.get_index_search()

forms_list = sorted(search.dbaccessor.read_all_indexreports_df().form.unique().tolist())

zip_reader = ZipReportReader.get_zip_by_name('2022q4.zip')
stmt_list = sorted(zip_reader.get_raw_pre_data().stmt.unique().tolist())

In [3]:
@interact(search_string=widgets.Text(value=''))
def search_cik(search_string):
    print(search_string)
    result_df = search.find_company_by_name(search_string)
    display(result_df)
    if len(result_df) > 0:
        print(result_df.cik.tolist()[0])

interactive(children=(Text(value='', description='search_string'), Output()), _dom_classes=('widget-interact',…

In [4]:
@interact
def reports(cik=widgets.Text(value='0'), forms=widgets.SelectMultiple(options=forms_list, rows=6, value=['10-K', '10-Q']), rows=[10, 25, 50, 100]):
    reader = CompanyReader.get_company_reader(cik=int(cik))
    reports = reader.get_all_company_reports_df(forms=list(forms))
    display(reports.head(rows))

interactive(children=(Text(value='0', description='cik'), SelectMultiple(description='forms', index=(4, 8), op…

In [ ]:
eigentlich wird im basereporting (oder dann wenigtans im reportreader) eine methode, welche nur merged, ohne filter auf datum oder so. leere columns mit NANs ollten dann einfach gelöscht werden können.

In [5]:
@interact
def reports(adsh=widgets.Text(value='0'), stmts=widgets.SelectMultiple(options=stmt_list, rows=6, value=['BS', 'IS', 'CF']), rows=[10, 25, 50, 100, 200]):
    reader = ReportReader.get_report_by_adsh(adsh=adsh)
    report_data = reader.financial_statements_for_period_and_previous_period()
    report_data = report_data[report_data.stmt.isin(stmts)]
    submission_data = {k:v for k,v in reader.submission_data().items() if k in ['cik', 'adsh', 'name', 'cityba', 'form', 'period', 'filed']}
    
    url = ReportZipIndexer.URL_PREFIX + str(submission_data['cik']) + '/' + submission_data['adsh'].replace('-','') + '/' + submission_data['adsh'] + '-index.htm'

    display(url)
    display(submission_data)
    # display(reader.statistics())
    display(report_data.head(rows))

interactive(children=(Text(value='0', description='adsh'), SelectMultiple(description='stmts', index=(0, 5, 1)…

In [9]:
[x for x in zip_reader.get_raw_pre_data().tag.unique().tolist() if "Cash" in str(x)]

['AccumulatedGainLossCashFlowHedgeIncludingNoncontrollingInterestMember',
 'AccumulatedGainLossNetCashFlowHedgeParentMember',
 'AccumulatedNetGainLossFromDesignatedOrQualifyingCashFlowHedgesMember',
 'AdditionalCashFlowElementsAbstract',
 'AdditionalCashFlowElementsAndSupplementalCashFlowInformationAbstract',
 'AdditionalCashFlowElementsFinancingActivitiesAbstract',
 'AdditionalCashFlowElementsInvestingActivitiesAbstract',
 'AdjustmentsNoncashItemsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivities',
 'AdjustmentsNoncashItemsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivitiesAbstract',
 'AdjustmentsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivities',
 'AdjustmentsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivitiesAbstract',
 'AssetRetirementObligationCashPaidToSettle',
 'BusinessCombinationRecognizedIdentifiableAssetsAcquiredAndLiabilitiesAssumedCashAndEquivalents',
 'CashAndCashEquivalentsAbstract',
 'CashAndCashEquivalentsAtCar